## Stub Plans 

### Links Úteis

- Documentação dos plan stubs: https://blueskyproject.io/bluesky/main/plans.html#stub-plans

- Documentação da customização de planos com per_step: https://blueskyproject.io/bluesky/main/plans.html#customize-step-scans-with-per-step

- Documentação da utilização de metadados: https://blueskyproject.io/bluesky/main/metadata.html

- Documentação dos preprocessors: https://blueskyproject.io/bluesky/main/plans.html#plan-preprocessors

In [110]:
%matplotlib qt
from time import time
from bluesky import RunEngine
from bluesky.plans import count
from bluesky.callbacks import LiveTable
from bluesky.plan_stubs import abs_set, rel_set, mv, mvr, \
    stage, unstage, stop, pause, deferred_pause, sleep, \
    checkpoint, clear_checkpoint, repeat
from ophyd import Device
from ophyd.sim import noisy_det, motor1, motor2
from matplotlib.pyplot import ion
from bluesky.utils import install_nb_kicker
install_nb_kicker()

ion()
RE = RunEngine()

#### Getting values from ophyd objects

In [ ]:
print(f"Read: {motor1.read()}")
print()
print(f"Configuration: {motor1.read_configuration()}")
print()
print(f"Get Device: {motor1.get()}")
print()
print(f"Get Readback: {motor1.readback.get()}")

## mv and abs_set

In [ ]:
#help(abs_set)
#help(mv)

In [ ]:
RE(mv(motor1, 0))
motor1.delay = 2
print(f"Before: {motor1.read()}")
RE(abs_set(motor1, 15))
print(f"After: {motor1.read()}")
motor1.delay = 0

In [ ]:
RE(mv(motor1, 0))
motor1.delay = 2
print(f"Before: {motor1.read()}")
RE(abs_set(motor1, 15, wait=True))
print(f"After: {motor1.read()}")
motor1.delay = 0

In [ ]:
RE(mv(motor1, 0, motor2, 0))
motor1.delay = 2
motor2.delay = 4
print(f"Before : {motor1.readback.get()}")
print(f"Before motor2: {motor1.readback.get()}")
start = time()
RE(mv(motor1, 4, motor2, 15))
end = time()
print(f"After motor1: {motor1.readback.get()}")
print(f"After motor2: {motor2.readback.get()}")
print(f"Duration: {end - start}")
motor1.delay = 0
motor2.delay = 0

## mvr and rel_set

In [ ]:
#help(mvr)
#help(rel_set)

In [ ]:
RE(mv(motor1, 14))
motor1.delay = 2
print(f"Before: {motor1.readback.get()}")
RE(rel_set(motor1, 15))
print(f"After: {motor1.readback.get()}")
motor1.delay = 0

In [ ]:
RE(mv(motor1, 14))
motor1.delay = 2
print(f"Before: {motor1.readback.get()}")
RE(mvr(motor1, 15))
print(f"After +15: {motor1.readback.get()}")
RE(mvr(motor1, -10))
print(f"After -10: {motor1.readback.get()}")
RE(mvr(motor1, 2.5))
print(f"After +2.5: {motor1.readback.get()}")
motor1.delay = 0

## sleep and stop

In [58]:
#help(sleep)
#help(stop)

In [63]:
class StopDevice(Device):

    def stop(self):
        super().stop()
        print("This Device has been stopped")

In [64]:
stop_device = StopDevice(name="stop_device")
RE(stop(stop_device))

This Device has been stopped


()

In [88]:
start = time()
RE(sleep(1))
end = time()
print(f"Duration: {end - start}")

Duration: 1.0067977905273438


## stage and unstage

In [26]:
#help(stage)
#help(unstage)

In [ ]:
class StagedDevice(Device):

    def stage(self):
        super().stage()
        print("This Device is being armed")

    def unstage(self):
        super().unstage()
        print("This Device is being restored to a safe idle state")

In [48]:
exampleDevice = StagedDevice(name="staged_device")

def stage_unstage_plan():
    yield from stage(exampleDevice)
    
    # Custom plan
    print("Custom plan")
    
    yield from unstage(exampleDevice)

RE(stage_unstage_plan())

This Device is being armed
Custom plan
This Device is being restored to a safe idle state


()

## Controlling the Run Engine 

In [91]:
#help(pause)
#help(deferred_pause)

In [93]:
#help(checkpoint)
#help(clear_checkpoint)

## repeat

In [96]:
help(repeat)

Help on function repeat in module bluesky.plan_stubs:

repeat(plan, num=1, delay=None)
    Repeat a plan num times with delay and checkpoint between each repeat.
    
    This is different from ``repeater`` and ``caching_repeater`` in that it
    adds ``checkpoint`` and optionally ``sleep`` messages if delay is provided.
    This is intended for users who need the structure of ``count`` but do not
    want to reimplement the control flow.
    
    Parameters
    ----------
    plan: callable
        Callable that returns an iterable of Msg objects
    num : integer, optional
        number of readings to take; default is 1
    
        If None, capture data until canceled
    delay : iterable or scalar, optional
        time delay between successive readings; default is 0
    
    Notes
    -----
    If ``delay`` is an iterable, it must have at least ``num - 1`` entries or
    the plan will raise a ``ValueError`` during iteration.



In [114]:
def repeat_count():
    yield from count([noisy_det], num=5, delay=0.25)
RE(repeat(repeat_count, num=3, delay=3), LiveTable(["noisy_det"]))



+-----------+------------+------------+
|   seq_num |       time |  noisy_det |
+-----------+------------+------------+
|         1 | 11:11:20.7 |      1.044 |
|         2 | 11:11:21.0 |      1.091 |
|         3 | 11:11:21.2 |      0.966 |
|         4 | 11:11:21.5 |      1.047 |
|         5 | 11:11:21.7 |      0.933 |
+-----------+------------+------------+
Plan count ['12c8e542'] (scan num: 7)




+-----------+------------+------------+
|   seq_num |       time |  noisy_det |
+-----------+------------+------------+
|         1 | 11:11:23.7 |      0.999 |
|         2 | 11:11:24.0 |      1.095 |
|         3 | 11:11:24.2 |      1.004 |
|         4 | 11:11:24.5 |      1.069 |
|         5 | 11:11:24.7 |      0.963 |
+-----------+------------+------------+
Plan count ['fa54bbf1'] (scan num: 8)




+-----------+------------+------------+
|   seq_num |       time |  noisy_det |
+-----------+------------+------------+
|         1 | 11:11:26.7 |      1.018 |
|         2 | 11:11:27.0 |      0.

('12c8e542-2f26-4830-951b-1e3d94d58f91',
 'fa54bbf1-e61b-479f-92c2-17f94591d2dd',
 'fe319c51-d77e-40a9-8a2e-3ad034af1476')

Fly Scan


Plans for asynchronous acquisition:

monitor: Asynchronously monitor for new values and emit Event documents.

unmonitor: Stop monitoring.

kickoff: Kickoff one fly-scanning device.

complete: Tell a flyable, 'stop collecting, whenever you are ready'.

collect: Collect data cached by one or more fly-scanning devices and emit documents.


Standard plans: per_step


Combinations of the above that are often convenient:

trigger_and_read(devices[, name])
	

Trigger and read a list of detectors and bundle readings into one Event.

one_1d_step(detectors, motor, step[, ...])
	

Inner loop of a 1D step scan

one_nd_step(detectors, step, pos_cache[, ...])
	

Inner loop of an N-dimensional step scan

one_shot(detectors[, take_reading])
	

Inner loop of a count.

move_per_step(step, pos_cache)
	

Inner loop of an N-dimensional step scan without any readings